# Web Scraping con Beautiful Soup

* * * 

### Iconos usados en este notebook
🔔 **Pregunta**: Una pregunta rápida para ayudarte a entender qué está pasando.<br>
🥊 **Desafío**: Ejercicio interactivo. ¡Lo resolveremos en el taller!<br>
⚠️ **Advertencia**: Atención sobre aspectos complicados o errores comunes.<br>
💡 **Consejo**: Cómo hacer algo de forma más eficiente o efectiva.<br>
🎬 **Demostración**: ¡Mostrando algo más avanzado para que sepas para qué se puede usar Python!<br>

### Objetivos de aprendizaje
1. [Reflexión: ¿Extraer o no extraer?](#when)
2. [Extraer y analizar HTML](#extract)
3. [Extraer la Asamblea General de Illinois](#scrape)

<a id='when'></a>

# Scrapear O No Scrapear

Cuando queremos acceder a datos de la web, primero debemos asegurarnos de que el sitio web que nos interesa ofrezca una API web. Plataformas como Twitter, Reddit y The New York Times ofrecen API. **Consulta las [API web de Python] de D-Lab.
(https://github.com/dlab-berkeley/Python-Web-APIs) si tu necesitas puedes aprender a usar APIs.**

Sin embargo, a menudo no existe una API web. En estos casos, podemos recurrir al web scraping, donde extraemos el HTML subyacente de una página web y obtenemos directamente la información deseada. Existen varios paquetes en Python que podemos usar para realizar estas tareas. Nos centraremos en dos paquetes: Requests y Beautiful Soup.

Nuestro estudio de caso recopilará información sobre los senadores estatales de Illinois (http://www.ilga.gov/senate), así como la lista de proyectos de ley patrocinados por cada senador. Antes de comenzar, revise estos sitios web para conocer su estructura.

## Instalacion

Usaremos dos paquetes principales: [Requisitos](http://docs.python-requests.org/en/latest/user/quickstart/) y [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Continúe instalando estos paquetes, si aún no lo ha hecho:

In [118]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [119]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


También instalaremos el paquete `lxml`, que ayuda a soportar parte del análisis que realiza Beautiful Soup:

In [120]:
%pip install requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [121]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [122]:
# Importacion de librerias requeridas
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracción y análisis de HTML


Para extraer y analizar HTML correctamente, seguiremos los siguientes 4 pasos:
1. Realizar una solicitud GET
2. Analizar la página con Beautiful Soup
3. Buscar elementos HTML
4. Obtener los atributos y el texto de estos elementos

## Paso 1: Realizar una solicitud GET para obtener el HTML de una página


Podemos usar la biblioteca de solicitudes para:

1. Realizar una solicitud GET a la página y
2. Leer el código HTML de la página web.

El proceso de realizar una solicitud y obtener un resultado es similar al del flujo de trabajo de la API web. Sin embargo, ahora realizamos una solicitud directamente al sitio web y tendremos que analizar el HTML nosotros mismos. Esto contrasta con recibir datos organizados en una salida JSON o XML más sencilla.

In [123]:
# Realizar una solicitud GET
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Leer el contenido de la respuesta del servidor
src = req.text
# Ver algunos resultados
print(src[:1000])

<html lang="en"> 
<!-- Trigger/Open The Modal -->
<div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
<button id="myBtn" style="color: white; background-color: navy; display: block">Translate Website</button></div>
<!-- The Modal -->
<div id="myModal" class="modal" style="display: none">
  <!-- Modal content -->
  <div class="modal-content">
      <div class="modal-header"><h3>
    <span class="close">&times;</span></h3></div>    
    <p>The Illinois General Assembly offers the Google Translate service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
    <p>Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
    <p>The English language version is always the official and authoritative version of this website.</p>
    <p>NOTE: To return to the original English language version, se


## Paso 2: Analiza la página con Beautiful Soup

Ahora, usamos la función `BeautifulSoup` para analizar la respuesta en un árbol HTML. Esto devuelve un objeto (llamado **objeto soup**) que contiene todo el HTML del documento original.

Si se produce un error relacionado con una biblioteca de análisis, asegúrese de haber instalado el paquete `lxml` para que Beautiful Soup cuente con las herramientas de análisis necesarias.

In [124]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

<html lang="en">
 <!-- Trigger/Open The Modal -->
 <body>
  <div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
   <button id="myBtn" style="color: white; background-color: navy; display: block">
    Translate Website
   </button>
  </div>
  <!-- The Modal -->
  <div class="modal" id="myModal" style="display: none">
   <!-- Modal content -->
   <div class="modal-content">
    <div class="modal-header">
     <h3>
      <span class="close">
       ×
      </span>
     </h3>
    </div>
    <p>
     The Illinois General Assembly offers the Google Translate service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.
    </p>
    <p>
     Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.
    </p>
    <p>
     The English language version is always the official and authoritative version of this website.
   

Ahora, usamos la función `BeautifulSoup` para analizar la respuesta en un árbol HTML. Esto devuelve un objeto (llamado **objeto soup**) que contiene todo el HTML del documento original.

El resultado es bastante similar al anterior, pero ahora está organizado en un objeto `soup`, lo que nos permite navegar por la página con mayor facilidad.

## Paso 3: Buscar elementos HTML


Beautiful Soup cuenta con varias funciones para encontrar componentes útiles en una página. Beautiful Soup permite encontrar elementos por:

1. Etiquetas HTML
2. Atributos HTML
3. Selectores CSS

Primero, busquemos **etiquetas HTML**.

La función `find_all` busca en el árbol `soup` todos los elementos con una etiqueta HTML específica y los devuelve.

¿Qué hace el siguiente ejemplo?

In [125]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>, <a href="/default.asp"><img alt="Illinois General Assembly" border="0" height="49" src="/images/logo_sm.gif" width="462"/></a>, <a class="mainmenu" href="/">Home</a>, <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>, <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>, <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseove

Dado que `find_all()` es el método más popular en la API de búsqueda de Beautiful Soup, puedes usar un atajo. Si tratas el objeto BeautifulSoup como si fuera una función, es lo mismo que llamar a `find_all()` en ese objeto.

Estas dos líneas de código son equivalentes:

In [126]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>
<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>


¿Cuántos enlaces obtuvimos?

In [127]:
print(len(a_tags))

213


¡Eso es muchísimo! Muchos elementos de una página tendrán la misma etiqueta HTML. Por ejemplo, si buscas todo con la etiqueta `a`, probablemente obtendrás más resultados, muchos de los cuales quizás no quieras. Recuerda que la etiqueta `a` define un hipervínculo, por lo que normalmente encontrarás muchos en cualquier página.

¿Qué sucedería si quisiéramos buscar etiquetas HTML con ciertos atributos, como clases CSS específicas?

Podemos hacerlo añadiendo un argumento adicional a `find_all`. En el siguiente ejemplo, buscamos todas las etiquetas `a` y luego las filtramos con `class_="sidemenu"`.

In [128]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>]

¡Eso es muchísimo! Muchos elementos de una página tendrán la misma etiqueta HTML. Por ejemplo, si buscas todo con la etiqueta `a`, probablemente obtendrás más resultados, muchos de los cuales quizás no desees. Recuerda que la etiqueta `a` define un hipervínculo, por lo que normalmente encontrarás muchos en cualquier página.

¿Qué sucedería si quisiéramos buscar etiquetas HTML con ciertos atributos, como clases CSS específicas?

Una forma más eficiente de buscar elementos en un sitio web es mediante un **selector CSS**. Para ello, debemos usar un método diferente llamado `select()`. Simplemente pasa una cadena a `.select()` para obtener todos los elementos con esa cadena como un selector CSS válido.

En el ejemplo anterior, podemos usar `"a.sidemenu"` como selector CSS, que devuelve todas las etiquetas `a` con la clase `sidemenu`.

In [129]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>]

## 🥊 Desafío: Encontrar todo

Usa BeautifulSoup para encontrar todos los elementos `a` con la clase `mainmenu`.

In [130]:
# Encontrar todos los elementos <a> con la clase 'mainmenu'
mainmenu_links = soup.find_all("a", class_="mainmenu")

# Mostrar los primeros 5 elementos encontrados
print(mainmenu_links[:5])

[<a class="mainmenu" href="/">Home</a>, <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>, <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>, <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseover="HM_f_PopUp('elMenu2',event)">House</a>, <a class="mainmenu" href="/mylegislation/" onblur="HM_f_PopDown('elMenu4')" onfocus="HM_f_PopUp('elMenu4',event)" onmouseout="HM_f_PopDown('elMenu4')" onmouseover="HM_f_PopUp('elMenu4',event)">My Legislation</a>]


## Paso 4: Obtener los atributos y el texto de los elementos

Una vez identificados los elementos, necesitamos la información de acceso de cada uno. Normalmente, esto implica dos cosas:

1. Texto
2. Atributos

Obtener el texto dentro de un elemento es sencillo. Solo tenemos que usar el miembro `text` de un objeto `tag`:

In [131]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.sidemenu")

# Examine the first link
first_link = side_menu_links[0]
print(first_link)

# What class is this variable?
print('Class: ', type(first_link))

<a class="sidemenu" href="/senate/default.asp">  Members  </a>
Class:  <class 'bs4.element.Tag'>


¡Es una etiqueta de Beautiful Soup! Esto significa que tiene un miembro "texto":

In [132]:
print(first_link.text)

  Members  


A veces necesitamos el valor de ciertos atributos. Esto es especialmente relevante para las etiquetas «a» o enlaces, donde el atributo «href» nos indica adónde lleva el enlace.

💡 **Consejo**: Puedes acceder a los atributos de una etiqueta tratándola como un diccionario:

In [133]:
print(first_link['href'])

/senate/default.asp


## 🥊 Desafío: Extraer atributos específicos

Extraer todos los atributos `href` de cada URL `mainmenu`.

In [134]:
# Encontrar todos los elementos <a> con la clase 'mainmenu'
mainmenu_links = soup.find_all("a", class_="mainmenu")

# Extraer los atributos 'href'
hrefs = [link.get("href") for link in mainmenu_links if link.get("href")]

# Mostrar los primeros 5 enlaces
print(hrefs[:5])

['/', '/legislation/', '/senate/', '/house/', '/mylegislation/']


<a id='scrape'></a>

# Análisis de la Asamblea General de Illinois

Aunque parezca increíble, estas son las herramientas fundamentales para analizar un sitio web. Una vez que dediques más tiempo a familiarizarte con HTML y CSS, solo tendrás que comprender la estructura de un sitio web específico y aplicar con inteligencia las herramientas de Beautiful Soup y Python.

Apliquemos estas habilidades para analizar la [98.ª Asamblea General de Illinois](http://www.ilga.gov/senate/default.asp?GA=98).

En concreto, nuestro objetivo es analizar la información de cada senador, incluyendo su nombre, distrito y partido.

## Analizar la página web

Analicemos la página web usando las herramientas que aprendimos en la sección anterior.

In [135]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Buscar los elementos de la tabla

Nuestro objetivo es obtener los elementos de la tabla en la página web. Recuerde: las filas se identifican con la etiqueta `tr`. Usemos `find_all` para obtener estos elementos.

In [136]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)

73

⚠️ **Advertencia**: Ten en cuenta que `find_all` obtiene *todos* los elementos con la etiqueta `tr`. Solo necesitamos algunos. Si usamos la función "Inspeccionar" de Google Chrome y observamos con atención, podemos usar selectores CSS para obtener solo las filas que nos interesan. En concreto, queremos las filas internas de la tabla:

In [137]:
# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

<tr><td colspan="5">
<span class="heading">Illinois State Senators</span>
<span class="italics">  98th  General Assembly</span><br/>
<!-- 3/2/09 temp comment out until fixed for GA specific-->
<!-- add 97th ga currently no info -->
<a href="98GA_Senate_Leadership.pdf">Leadership</a> <a href="98th_Senate_Officers.pdf">Officers</a> <a href="98GA_Senate_Seating_Chart.pdf">Senate Seating Chart</a>  <span class="content"><b>Democrats:</b> 40   <b>Republicans:</b> 19</span><br/>
</td></tr> 

<tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',98);" title="Sort by Senator">Senator</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',98);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('Party','

Parece que queremos todo lo que queda después de las dos primeras filas. Empecemos con una sola fila y construyamos nuestro bucle a partir de ahí.

In [138]:
example_row = rows[2]
print(example_row.prettify())

<tr>
 <td bgcolor="white" class="detail" width="40%">
  <a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">
   Pamela J. Althoff
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="SenatorBills.asp?GA=98&amp;MemberID=1911">
   Bills
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="SenCommittees.asp?GA=98&amp;MemberID=1911">
   Committees
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  32
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  R
 </td>
</tr>



Desglosemos esta fila en sus celdas/columnas mediante el método `select` con selectores CSS. Si analizamos el HTML con atención, hay un par de maneras de hacerlo.

* Podríamos identificar las celdas por su etiqueta `td`.
* Podríamos usar el nombre de clase `.detail`.
* Podríamos combinar ambos y usar el selector `td.detail`.

In [139]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>
<td align="center" bgcolor="white" class="detail" width="15%">32</td>
<td align="center" bgcolor="white" class="detail" width="15%">R</td>

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>
<td align="center" bgcolor="white" class

Podemos confirmar que todos son iguales.

In [140]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Utilicemos el selector `td.detail` para ser lo más específicos posible.

In [141]:
# Select only those 'td' tags with class 'detail' 
detail_cells = example_row.select('td.detail')
detail_cells

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%">32</td>,
 <td align="center" bgcolor="white" class="detail" width="15%">R</td>]

La mayoría de las veces, nos interesa el **texto** real de un sitio web, no sus etiquetas. Recordemos que para obtener el texto de un elemento HTML, usamos el miembro `text`:

In [142]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

['Pamela J. Althoff', 'Bills', 'Committees', '32', 'R']


¡Se ve bien! Ahora solo necesitamos usar nuestros conocimientos básicos de Python para obtener los elementos de esta lista que necesitamos. Recuerda: queremos el nombre del senador, su distrito y su partido.

In [143]:
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

Pamela J. Althoff
32
R


## Eliminando filas basura

Vimos al principio que no todas las filas que obtuvimos corresponden a un senador. Tendremos que hacer limpieza antes de continuar. Vean algunos ejemplos:

In [144]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

Row 0:
 <tr><td colspan="5">
<span class="heading">Illinois State Senators</span>
<span class="italics">  98th  General Assembly</span><br/>
<!-- 3/2/09 temp comment out until fixed for GA specific-->
<!-- add 97th ga currently no info -->
<a href="98GA_Senate_Leadership.pdf">Leadership</a> <a href="98th_Senate_Officers.pdf">Officers</a> <a href="98GA_Senate_Seating_Chart.pdf">Senate Seating Chart</a>  <span class="content"><b>Democrats:</b> 40   <b>Republicans:</b> 19</span><br/>
</td></tr> 

Row 1:
 <tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',98);" title="Sort by Senator">Senator</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',98);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascrip

Al escribir nuestro bucle for, queremos que solo se aplique a las filas relevantes. Por lo tanto, debemos filtrar las filas irrelevantes. Para ello, comparamos algunas de estas filas con las que necesitamos, observamos sus diferencias y luego formulamos esto en una condición.

Como puedes imaginar, hay muchas maneras de hacerlo, y dependerá del sitio web. Aquí te mostraremos algunas para que te hagas una idea de cómo hacerlo.

In [145]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

1
11
5
5


Quizás las buenas filas tengan una longitud de 5. Comprobémoslo:

In [146]:
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td><td align="center" bgcolor="white" class="detail" width="15%">32</td><td align="center" bgcolor="white" class="detail" width="15%">R</td></tr> 

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=2035">Patricia Van Pelt</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=2035">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=2035">Committees</a></td><td align="center" bgcolor="white

Encontramos una fila de pie de página en nuestra lista que queremos evitar. Probemos algo diferente:

In [147]:
rows[2].select('td.detail') 

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%">32</td>,
 <td align="center" bgcolor="white" class="detail" width="15%">R</td>]

In [148]:
# Bad row
print(rows[-1].select('td.detail'), '\n')

# Good row
print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

[] 

[<td bgcolor="EBEBEB" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=2022">Jennifer Bertino-Tarrant</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=2022">Bills</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=2022">Committees</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%">49</td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%">D</td>] 

Checking rows...

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></

¡Parece que encontramos algo que funcionó!

## Unir todo en un bucle

Ahora que hemos visto cómo obtener los datos que queremos de una fila y filtrar los que no necesitamos, vamos a unirlo todo en un bucle.

In [149]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

In [150]:
# Should be 61
len(members)

61

Echemos un vistazo a lo que tenemos en "miembros".

In [151]:
print(members[:5])

[('Pamela J. Althoff', 32, 'R'), ('Jason A. Barickman', 53, 'R'), ('Scott M Bennett', 52, 'D'), ('Jennifer Bertino-Tarrant', 49, 'D'), ('Daniel Biss', 9, 'D')]


## 🥊  Desafío: Obtener elementos `href` que apunten a los proyectos de ley de los miembros

El código anterior recupera información sobre:

- el nombre del senador,
- su número de distrito,
- y su partido.

Ahora queremos recuperar la URL de la lista de proyectos de ley de cada senador. Cada URL seguirá un formato específico.

El formato de la lista de proyectos de ley de un senador determinado es:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

para obtener algo como:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

donde `MEMBER_ID=1911`.

Deberías poder ver que, lamentablemente, `MEMBER_ID` no se extrae actualmente en nuestro código de extracción.

Tu tarea inicial es modificar el código anterior para que también **recuperemos la URL completa que apunta a la página correspondiente de los proyectos de ley patrocinados por las primarias**, para cada miembro, y la devolvamos junto con su nombre, distrito y partido.

Consejos:

* Para ello, deberás obtener el elemento de anclaje apropiado (`<a>`) en la fila de la tabla de cada legislador. Puedes usar el método `.select()` en el objeto `row` del bucle para hacerlo, similar al comando que encuentra todas las celdas `td.detail` de la fila. Recuerda que solo queremos el enlace a los proyectos de ley del legislador, no a los comités ni a su página de perfil.
* El HTML de los elementos de anclaje se verá como `<a href="/senate/Senator.asp/...">Proyectos de ley</a>`. La cadena del atributo `href` contiene el enlace **relativo** que buscamos. Puedes acceder a un atributo de un objeto `Tag` de BeatifulSoup de la misma manera que accedes a un diccionario de Python: `anchor['attributeName']`. Consulta la <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentación</a> para más detalles.
* Hay muchas maneras diferentes de usar BeautifulSoup. Puedes hacer lo que necesites para extraer el `href`.

El código se ha completado parcialmente. Complétalo donde dice `#TU CÓDIGO AQUÍ`. Guarda la ruta en un objeto llamado `full_path`.

In [152]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [153]:
# Hacer una solicitud GET a la página del Senado de Illinois
url = 'http://www.ilga.gov/senate/default.asp'
req = requests.get(url)

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(req.text, 'lxml')

# Encontrar la tabla de senadores
senator_rows = soup.select("table[class='senate'] tr")

# Lista para almacenar la información
senators_info = []

# Iterar sobre las filas de la tabla
for row in senator_rows:
    # Extraer los datos de la fila
    cells = row.select("td.detail")
    
    if len(cells) >= 3:  # Verificar si la fila tiene datos válidos
        name = cells[0].text.strip()
        district = cells[1].text.strip()
        party = cells[2].text.strip()
        
        # Encontrar el enlace a los proyectos de ley del senador
        bill_link = row.select_one("a[href*='SenatorBills.asp']")
        
        if bill_link:
            relative_path = bill_link["href"]  # Obtener el 'href' relativo
            full_path = f"http://www.ilga.gov{relative_path}"  # Construir la URL completa
        else:
            full_path = "No disponible"  # En caso de que no haya enlace

        # Agregar los datos a la lista
        senators_info.append((name, district, party, full_path))

# Mostrar los primeros 5 resultados
for senator in senators_info[:5]:
    print(senator)

## 🥊  Desafío: Modulariza tu código

Convierte el código anterior en una función que acepte una URL, rastree la URL en busca de sus senadores y devuelva una lista de tuplas con información sobre cada senador.

In [205]:
def obtener_senadores(url):
    # Hacer una solicitud GET a la página
    print(f"Obteniendo senadores de {url}...")
    req = requests.get(url)

    # Verificar si la página respondió correctamente
    if req.status_code != 200:
        print(f"Error: No se pudo acceder a la página (Código {req.status_code})")
        return []

    # Parsear el HTML con BeautifulSoup
    soup = BeautifulSoup(req.text, 'lxml')

    # Encontrar todas las filas de la tabla de senadores
    senator_rows = soup.select("table tr")

    # Lista para almacenar la información
    senators_info = []

    # Verificar si se encontraron filas
    if not senator_rows:
        print("No se encontraron filas de senadores.")
        return []

    # Iterar sobre las filas de la tabla
    for row in senator_rows:
        # Extraer los datos de la fila
        cells = row.select("td.detail")
        
        # Verificar si la fila tiene datos válidos (mínimo 3 celdas)
        if len(cells) >= 3:
            name = cells[0].text.strip()
            district = cells[1].text.strip()
            party = cells[2].text.strip()
            
            # Buscar el enlace a los proyectos de ley del senador
            bill_link = row.select_one("a[href*='SenatorBills.asp']")
            
            if bill_link:
                relative_path = bill_link["href"]  # Obtener el 'href' relativo
                full_path = f"http://www.ilga.gov/senate/{relative_path}"  # Construir la URL completa
            else:
                full_path = "No disponible"

            # Agregar la información a la lista
            senators_info.append((name, district, party, full_path))

    # Si la lista está vacía, informar
    if not senators_info:
        print("No se encontraron senadores en la tabla.")

    return senators_info

In [231]:
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = obtener_senadores(url)

print(f"Cantidad de senadores encontrados: {senate_members}\n")
# Imprimir los primeros 5 senadores obtenidos
for senador in senate_members[:50]:
    print(senador)


Obteniendo senadores de http://www.ilga.gov/senate/default.asp?GA=98...
Cantidad de senadores encontrados: [('Pamela J. Althoff', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=1911'), ('Pamela J. Althoff', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=1911'), ('Pamela J. Althoff', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=1911'), ('Jason A. Barickman', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2018'), ('Scott M Bennett', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2272'), ('Jennifer Bertino-Tarrant', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2022'), ('Daniel Biss', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2020'), ('Tim Bivins', 'Bills', 'Committees', 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=194

## 🥊 Desafío práctico: Escribir una función de scraping

Queremos scraping las páginas web correspondientes a los proyectos de ley patrocinados por cada proyecto de ley.

Escribe una función llamada `get_bills(url)` para analizar la URL de un proyecto de ley. Esto implica:

- Solicitar la URL mediante la biblioteca <a href="http://docs.python-requests.org/en/latest/">`requests`</a>
- Usar las funciones de la biblioteca `BeautifulSoup` para encontrar todos los elementos `<td>` con la clase `billlist`
- Devolver una _lista_ de tuplas, cada una con:
- Descripción (2.ª columna)
- Cámara (S o H) (3.ª columna)
- La última acción (4.ª columna)
- La fecha de la última acción (5.ª columna)

Esta función se ha completado parcialmente. Completa el resto.

In [232]:
def get_bills(url):
    # Obtener el contenido HTML de la página
    src = requests.get(url).text
    soup = BeautifulSoup(src, 'lxml')  # Asegúrate de usar el parser correcto
    
    # Buscar todas las filas de la tabla
    rows = soup.select('tr')
    bills = []

    for row in rows:
        # Seleccionar todas las celdas con clase 'billlist'
        cells = row.select('td.billlist')
        
        # Verificamos que haya al menos 5 celdas (las columnas necesarias)
        if len(cells) >= 5:
            bill_id = cells[0].text.strip()
            description = cells[1].text.strip()
            chamber = cells[2].text.strip()
            last_action = cells[3].text.strip()
            last_action_date = cells[4].text.strip()
            
            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)

    return bills


In [236]:
test_url = senate_members[0][3]
bills = get_bills(test_url)
for bill in bills[:50]:
    print(bill)


('SB2', 'STATE GOVERNMENT-TECH', 'S', 'Session Sine Die', '1/13/2015')
('SB2', 'STATE GOVERNMENT-TECH', 'S', 'Session Sine Die', '1/13/2015')
('SB2', 'STATE GOVERNMENT-TECH', 'S', 'Session Sine Die', '1/13/2015')
('SB9', 'PUBLIC UTIL-PERFORMANCE-BASED', 'S', 'Public Act . . . . . . . . . 98-0015', '5/23/2013')
('SB27', 'MEDICAID BUDGET NOTE ACT', 'S', 'Session Sine Die', '1/13/2015')
('SB28', 'HOMELESS VETERANS SHELTER ACT', 'S', 'Session Sine Die', '1/13/2015')
('SB29', 'ROAD FUND-NO TRANSFERS', 'S', 'Session Sine Die', '1/13/2015')
('SB31', 'UNIFORM COLLABORATIVE LAW ACT', 'S', 'Session Sine Die', '1/13/2015')
('SB33', 'EPA-RULES-DOCUMENT SUBMISSION', 'S', 'Public Act . . . . . . . . . 98-0072', '7/15/2013')
('SB44', '$DHS- MENTAL HEALTH PROGRAMS', 'S', 'Session Sine Die', '1/13/2015')
('SB62', 'MHDD CD-CLINICAL PSYCHOLOGIST', 'S', 'Public Act . . . . . . . . . 98-0075', '7/15/2013')
('SB104', 'MIN WAGE-OVERTIME-ALTERN SHIFT', 'S', 'Session Sine Die', '1/13/2015')
('SB124', 'GOVERNME

### Extraer todos los proyectos de ley

Finalmente, cree un diccionario `bills_dict` que asigne un número de distrito (la clave) a una lista de proyectos de ley (el valor) provenientes de ese distrito. Puede hacerlo recorriendo en bucle todos los miembros del senado en `members_dict` y llamando a `get_bills()` para cada una de las URL de sus proyectos de ley asociados.

**NOTA:** Por favor, llame a la función `time.sleep(1)` en cada iteración del bucle para no destruir el sitio web del estado.

In [237]:
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = obtener_senadores(url)

members_dict = {senador[1]: senador[3] for senador in senate_members if senador[3] != "No disponible"}
print(members_dict)


Obteniendo senadores de http://www.ilga.gov/senate/default.asp?GA=98...
{'Bills': 'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2035'}


In [ ]:
import json

# Guardar como archivo JSON
with open("proyectos_por_distrito.json", "w", encoding="utf-8") as f:
    json.dump(bills_dict, f, ensure_ascii=False, indent=2)

print("Archivo guardado como 'proyectos_por_distrito.json'")


✅ Archivo guardado como 'proyectos_por_distrito.json'
